## Imports

In [10]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import math
import re

from util.word2vec_as_MF import Word2vecMF
from util.functions import *

import time
dimension = 100
regularization =0.0
model_enwik = Word2vecMF()
negative = 5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read and preprocess enwik9

# Gensim

In [11]:
'''%%time
skip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'''

'%%time\nskip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'

In [12]:
'''skip.get_latest_training_loss()'''

'skip.get_latest_training_loss()'

## Train ro_sgns model starting from SVD of SPPMI

In [13]:
%%time
# Create word2vobject has no attributeec as matrix factorization model

'''model_enwik.data_to_matrices(real_sentences, dimension, 5, 
                             DB_to_file='enwik-200/matrices.npz',
                            indices_to_file='enwik-200/vocab.txt')'''
#sentences = load('data/x1')

model_enwik.data_to_matrices('data/x2', r=dimension, k=negative, 
                             DB_to_file=False,
                            vocab_to_file=False)

Parsing sentences from training set
CPU times: user 2.27 s, sys: 44.4 ms, total: 2.32 s
Wall time: 2.49 s


In [14]:
#model_enwik.load_matrices('data/x1.npz')


In [15]:
#C0, W0, step_size = BFGD_init(model_enwik, dimension=dimension, reg=regularization)
C0, W0 = SPPMI_init(model_enwik, dimension=dimension, negative = negative)
#C0, W0, step_size = RAND_init(model_enwik, dimension=dimension)
print(np.count_nonzero(C0))

/home/yerong/local/Anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


0.05626258087294637
218.6681790265787
('Initial loss', 6259647.136484)
93400


In [16]:
print(C0.shape[1])
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='AM', 
               d=dimension,
               eta = 5e-6,
               lbd = 1.0,
               MAX_ITER=189000,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

934


'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'AM\', \n               d=dimension,\n               eta = 5e-6,\n               lbd = 1.0,\n               MAX_ITER=189000,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [17]:
start_time = time.time()
opt_experiment(model_enwik,
               mode='BFGD',
               d=dimension,
               eta=1e-5,
               #eta = step_size,
               reg = regularization,
               MAX_ITER=50,
               from_iter=0,
               start_from='test',
               itv_print=1,
               itv_save=1,
               init=(True, C0, W0), display=True,
               autostop=False)
print("--- %s seconds ---" % (time.time() - start_time))

enwik-200/BFGDiter_fromtest_dim100_step1e-05_0.0
('Iter #:', 0, 'loss', 6259647.136484)
('Iter #:', 1, 'loss', 6243014.10426398)
('Iter #:', 2, 'loss', 6231307.333076783)
('Iter #:', 3, 'loss', 6207090.88606139)
('Iter #:', 4, 'loss', 6141099.284061376)
('Iter #:', 5, 'loss', 5997933.6603699075)
('Iter #:', 6, 'loss', 5815760.463274696)
('Iter #:', 7, 'loss', 5665073.119794607)
('Iter #:', 8, 'loss', 5543744.169748818)
('Iter #:', 9, 'loss', 5439249.8341753185)
('Iter #:', 10, 'loss', 5347175.55540183)
('Iter #:', 11, 'loss', 5265094.0506687695)
('Iter #:', 12, 'loss', 5190628.541480141)
('Iter #:', 13, 'loss', 5121638.3497929685)
('Iter #:', 14, 'loss', 5056497.056384536)
('Iter #:', 15, 'loss', 4994107.764759321)
('Iter #:', 16, 'loss', 4933819.899752124)
('Iter #:', 17, 'loss', 4875351.189228394)
('Iter #:', 18, 'loss', 4818735.273569775)
('Iter #:', 19, 'loss', 4764265.287300496)
('Iter #:', 20, 'loss', 4712394.958796121)
('Iter #:', 21, 'loss', 4663587.782789797)
('Iter #:', 22, '